In [8]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

Dog1 = Image.open("Img/Dog1.jpg")
Dog1 = Dog1.resize((64, 64))
Dog1_array = np.array(Dog1)
Dog1_array = Dog1_array / 250
Dog2 = Image.open("Img/Dog2.jpg")
Dog2 = Dog2.resize((64, 64))
Dog2_array = np.array(Dog2)
Dog2_array = Dog2_array / 250
Dog3 = Image.open("Img/Dog3.jpg")
Dog3 = Dog3.resize((64, 64))
Dog3_array = np.array(Dog3)
Dog3_array = Dog3_array / 250
Cat1 = Image.open("Img/Cat1.jpeg")
Cat1 = Cat1.resize((64, 64))
Cat1_array = np.array(Cat1)
Cat1_array = Cat1_array / 250
Cat2 = Image.open("Img/Cat2.jpg")
Cat2 = Cat2.resize((64, 64))
Cat2_array = np.array(Cat2)
Cat2_array = Cat2_array / 250
Cat3 = Image.open("Img/Cat3.jpg")
Cat3 = Cat3.resize((64, 64))
Cat3_array = np.array(Cat3)
Cat3_array = Cat3_array / 250

#Nos falta aplanar las imagenes en un vector (flatten)


